In [ ]:
import itertools as it
import operator
import re
from collections import Counter
from pathlib import Path

import duckdb
import gfapy
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from pyarrow import csv
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.io as sio

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

# Config

In [ ]:
data_dir = Path(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/"
)

In [ ]:
arrow_ds = ds.dataset(list(data_dir.glob("*.arrow")), format="arrow")
arrow_ds2 = ds.dataset(list(data_dir.glob("*.arrow"))[:1], format="arrow")
parquet_ds = ds.dataset(list(data_dir.glob("*.parquet")), format="parquet")

# Group by path

## Setup

In [ ]:
gfa = gfapy.Gfa.from_file(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/barcode.gfa"
)

In [ ]:
bc_segments_oriented = [
    [f"{o}{s}" for s in gfa.segment_names if s.startswith("BC:")] for o in "><"
]

In [ ]:
bc_segments = [
    f"{o}{s}" for s in gfa.segment_names if s.startswith("BC:BIT") for o in "><"
]

In [ ]:
reverse_path_mapping = {
    f"{o[0]}{s}": f"{o[1]}{s}" for s in gfa.segment_names for o in ["<>", "><"]
}

## Polars

In [ ]:
def consensus_func(df):
    return (
        df.with_columns(
            pl.col("name").str.contains(";").alias("is_duplex"),
            pl.col("name").str.contains(";").not_().alias("is_simplex"),
        )
        .with_columns(
            pl.sum("is_simplex").alias("depth_simplex"),
            pl.sum("is_duplex").alias("depth_duplex"),
        )
        .select(
            pl.col(
                "path_subset",
                "read_seq",
                "read_phred",
                "depth_simplex",
                "depth_duplex",
            )
        )
        .head(1)
    )

In [ ]:
%%time
with pl.StringCache():
    df = pl.scan_pyarrow_dataset(arrow_ds2)
    res = (
        df.with_columns(  # .limit(100_000)
            pl.col("path").list.set_intersection(bc_segments)
            # TODO: waiting on https://github.com/pola-rs/polars/issues/11735
            # to keep path columns as categorical
            # .cast(pl.List(pl.Categorical))
            .alias("_path_subset"),
        )
        .with_columns(
            pl.col("_path_subset")
            .list.reverse()
            .list.eval(pl.element().map_dict(reverse_path_mapping))
            .alias("_path_subset_reversed"),
            (
                pl.col("path").list.set_intersection(bc_segments_oriented[0]).list.len()
                > 0
            ).alias("_is_forward"),
            (
                pl.col("path").list.set_intersection(bc_segments_oriented[1]).list.len()
                > 0
            ).alias("_is_reverse"),
        )
        .with_columns(
            pl.when(pl.col("_is_forward") & pl.col("_is_reverse").not_())
            .then(False)
            .when(pl.col("_is_forward").not_() & pl.col("_is_reverse"))
            .then(True)
            .otherwise(None)
            .alias("reverse_complement"),
        )
        .with_columns(
            pl.when(pl.col("reverse_complement") == False)
            .then(pl.col("_path_subset"))
            .when(pl.col("reverse_complement") == True)
            .then(pl.col("_path_subset_reversed"))
            .cast(pl.List(pl.Categorical))
            .alias("path_subset")
        )
        .filter(pl.col("path_subset").hash() % 1000 == 3)
        .filter(pl.col("path_subset").list.len() == 30)
        .select(
            pl.col(
                "name",
                "read_seq",
                "read_phred",
                "path",
                "path_subset",
                "reverse_complement",
            )
        )
        .group_by("path_subset")
        .agg(pl.col("name", "read_seq", "read_phred"))
        # .map_groups(
        #     consensus_func,
        #     schema=dict(
        #         path_subset=pl.List(pl.Categorical),
        #         read_seq=pl.Utf8,
        #         read_phred=pl.Utf8,
        #         depth_simplex=pl.UInt64,
        #         depth_duplex=pl.UInt64,
        #     ),
        # )
    ).collect()
# res

In [ ]:
test_reads = res

In [ ]:
test_reads.select(pl.col("name")).with_columns(pl.col("name").list.len()).sort("name")

## DuckDB

In [ ]:
%%time
res = duckdb.execute(
    "SELECT path, list_intersect(path, $bc_segments) FROM arrow_ds2 LIMIT 2",
    dict(bc_segments=bc_segments),
).arrow()

In [ ]:
res